# Stepping Through a Pyteller Pipeline

In [1]:
from pyteller.data import load_data

 
## Load the input Data
Input data is loaded from s3 bucket

In [2]:
current_data,input_data = load_data('AL_Weather')
current_data.head()

,station,valid,tmpf,dwpf,relh,drct,sknt,p01i,alti,vsby,feel
0,8A0,1/1/16 0:15,41.0,39.2,93.24,350,6,0.0,30.25,10.0,36.37
1,4A6,1/1/16 0:15,41.0,32.0,70.08,360,5,0.0,30.30,10.0,37.08
2,8A0,1/1/16 0:35,39.2,37.4,93.19,360,6,0.0,30.25,10.0,34.20
3,4A6,1/1/16 0:35,41.0,32.0,70.08,360,5,0.0,30.29,10.0,37.08
4,8A0,1/1/16 0:55,37.4,37.4,100.00,360,8,0.0,30.25,10.0,30.76


In [3]:
from mlblocks.discovery import find_pipelines
find_pipelines('pyteller')

['pyteller.ARIMA.arima',
 'pyteller.LSTM.LSTM_offset',
 'pyteller.persistence.persistence',
 'pyteller.persistence.persistence_step_through']

In [4]:
pipeline= 'pyteller.persistence.persistence_step_through'

In [5]:
from pyteller.core import Pyteller
pyteller = Pyteller(
    pipeline=pipeline,
    pred_length=10,
    offset=5,
    time_column='valid',
    target_signal='tmpf',
    entity_column='station',
    entities=['8A0']
)

In [6]:
context = pyteller.fit(current_data, output_=0)
train = context['X']
train.head()

entity,timestamp,8A0
0,1.451607e+09,41.0
1,1.451608e+09,39.2
2,1.451610e+09,37.4
33,1.451611e+09,37.4
34,1.451612e+09,37.4


## Finish fitting the pipeline

In [7]:
pyteller.fit(start_=1, **context)


## Forecast

In [8]:
output = pyteller.forecast(data=input_data, postprocessing=False, predictions_only=False)
output['forecast'].head()

entity,8A0
2016-01-29 03:35:00,44.60
2016-01-29 03:55:00,44.60
2016-01-29 04:15:00,44.60
2016-01-29 04:35:00,44.15
2016-01-29 04:55:00,43.88


## Evaluate

In [9]:
scores = pyteller.evaluate(test_data=output['actuals'],forecast=output['forecast'],
                           metrics=['MAPE','sMAPE'])
scores.head()

entity,8A0
sMAPE,8.984589
MAPE,8.967387


## Plot

In [10]:
pyteller.plot(output)